In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import logging
import sys
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import json

scripts_dir = Path("../..").joinpath("src")
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())

from helpers.strapi import Strapi
from helpers.settings import get_settings, Settings
from helpers.file_handler import FileConventionHandler
from helpers.utils import download_and_unzip_if_needed, writeReadGCP

from pipelines.output_schemas import (
    FPLSchema,
    ProtectionLevelSchema,
    MPAsSchema,
    HabitatsSchema,
    LocationSchema,
    ProtectedAreaExtentSchema,
)
from pipelines.processors import (
    add_envelope,
    add_location_iso,
    expand_multiple_locations,
    add_region_iso,
    calculate_eez_area,
    add_bbox,
    add_groups_and_members,
    add_location_name,
    output,
    clean_geometries,
    filter_by_exluding_propossed_mpas,
    spatial_join,
    process_mpa_data,
    assign_iso3,
    calculate_global_area,
    separate_parent_iso,
    calculate_stats_cov,
    coverage_stats,
    mpaatlas_filter_stablishment,
    process_mpaatlas_data,
    calculate_stats,
    fix_monaco,
    batch_export,
    calculate_area,
    define_is_child,
    set_child_id,
    add_child_parent_relationship,
    columns_to_lower,
    extract_wdpaid_mpaatlas,
)

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("fiona").setLevel(logging.WARNING)

In [3]:
mysettings = get_settings()
prev_step = "preprocess"
current_step = "stats"

In [125]:
# Strapi setup
strapi = Strapi(url=mysettings.STRAPI_URL)
strapi.login(jwt=mysettings.STRAPI_JWT)

In [4]:
# after generated the locations file for the first time:
location_code = pd.read_csv(mysettings.DATA_DIR.joinpath("eez/processed/stats/locations_code.csv"))

### Locations (eez + regions)

In [7]:
pipe = "eez"
strapi_collection = ""
pipe_dir = FileConventionHandler(pipe)

output_file = pipe_dir.get_processed_step_path(current_step).joinpath("locations.json")

# Download the EEZ file && unzip it
download_and_unzip_if_needed(pipe_dir, prev_step, mysettings)

DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token


/home/mambauser/data/eez/processed/eez_preprocess.zip
/home/mambauser/data/eez/processed/preprocess


PosixPath('/home/mambauser/data/eez/processed/preprocess')

In [8]:
locations = (
    gpd.read_file(pipe_dir.get_step_fmt_file_path(prev_step, "shp"))
    .pipe(add_envelope)
    .pipe(add_location_iso)
    .pipe(expand_multiple_locations)
    .pipe(add_region_iso,'iso')
    .pipe(calculate_eez_area)
    .pipe(add_bbox)
    .pipe(add_groups_and_members)
    .pipe(add_location_name)
    .rename(
        columns={
            "iso": "code",
            "AREA_KM2": "totalMarineArea",
            "location_type": "type",
        }
    )
)

locations.drop(
    columns=list(
        set(locations.columns) - 
        set(["code", "name", "totalMarineArea", "type", "groups", "bounds", "id"])),
    inplace=True,
)

output_locations = {
    "version": 2,
    "data": {
        "api::location.location": LocationSchema(pd.DataFrame(locations)).to_dict(
            orient="index"
        )
    },
}
with open(output_file, "w") as f:
    json.dump(output_locations, f)

del output_locations

In [9]:
##
(locations[['id', 'code']]
 .to_csv(pipe_dir.get_processed_step_path(current_step)
     .joinpath('locations_code.csv'), index=False))

del locations

In [ ]:
strapi.deleteCollectionData(strapi_collection, list(range(1, 300)))

In [ ]:
strapi.importCollectionData(
    strapi_collection,
    output_file,
)

### Habitats

The habitat data came from:


In [8]:
pipe = "habitats"
strapi_collection = ''
pipe_dir = FileConventionHandler(pipe)
output_file = pipe_dir.get_step_fmt_file_path(current_step, "csv")

# Download the habitat file
# download_and_unzip_if_needed(pipe_dir, prev_step, mysettings)

habitats_intermediate = pd.read_csv(
    pipe_dir.processed_path.joinpath("habitats4.csv"), keep_default_na=False
)

In [ ]:
habitat_stats = habitats_intermediate.pipe(
    output,
    iso_column="location_id",
    rep_d={
        "habitat_name": {
            "saltmarshes": 1,
            "seagrasses": 2,
            "warm-water corals": 3,
            "cold-water corals": 4,
            "mangroves": 5,
            "seamounts": 6,
        },
        "protected_area": {"": 0},
    },
    rename={
        "protected_area": "protectedArea",
        "total_area": "totalArea",
        "habitat_name": "habitat",
    },
    drop_cols=["location_id"],
)
HabitatsSchema(habitat_stats).to_csv(
    output_file.as_posix(), index=True
)

In [ ]:
strapi.deleteCollectionData(strapi_collection, list(range(1, 300)))

In [ ]:
strapi.importCollectionData(
    strapi_collection,
    output_file,
)

### Coverage stats - Mpas

We are going to use the intermediate data from eez, in order to create a dataset that can be used as a land mask.
The steps are:
1. Load eez
2. Spatial inner Join the eez dataset with the Mpas one
3. Assign the location iso
4. dissolve by location iso and cummulative year
5. calculate the area for global regions and eez countries
6. prepare the data to be ingested in strapi
7. upload the data to strapi

In [5]:
pipe = "mpa"
strapi_collection = ""

pipe_dir_eez = FileConventionHandler("eez")
pipe_dir_mpas = FileConventionHandler(pipe)
output_file = pipe_dir_mpas.get_processed_step_path(current_step).joinpath(
    "mpa_landmask_strapi.csv"
)

# Download the EEZ file && unzip it
download_and_unzip_if_needed(pipe_dir_eez, prev_step, mysettings)
# Download the mpas file && unzip it
download_and_unzip_if_needed(pipe_dir_mpas, prev_step, mysettings)

# Load the data
eez = gpd.read_file(pipe_dir_eez.get_step_fmt_file_path(prev_step, "shp")).pipe(clean_geometries)
mpas = gpd.read_file(pipe_dir_mpas.get_step_fmt_file_path(prev_step, "shp")).pipe(clean_geometries)

/home/mambauser/data/eez/processed/eez_preprocess.zip
/home/mambauser/data/eez/processed/preprocess
/home/mambauser/data/mpa/processed/mpa_preprocess.zip
/home/mambauser/data/mpa/processed/preprocess


In [6]:
eez_mpas_data_join = await spatial_join(eez, mpas.pipe(filter_by_exluding_propossed_mpas))

100%|█████████▉| 281/282 [06:12<01:03, 63.36s/it]

<class 'shapely.geometry.base.GeometrySequence'>


100%|██████████| 282/282 [06:33<00:00,  1.39s/it]

<class 'shapely.geometry.base.GeometrySequence'>
<class 'shapely.geometry.base.GeometrySequence'>
<class 'shapely.geometry.base.GeometrySequence'>
<class 'shapely.geometry.base.GeometrySequence'>


In [7]:
# To get an idea of the spatial join results
eez_mpas_data_join.pipe(add_location_iso).pipe(assign_iso3).to_file(
    pipe_dir_mpas.get_processed_step_path(current_step).joinpath("mpas_sjoin.shp"), driver="ESRI Shapefile"
)

In [9]:
final_data = await process_mpa_data(
    eez_mpas_data_join.pipe(add_location_iso).pipe(assign_iso3),
    range(2011, time.localtime().tm_year + 1),
    ["PA_DEF", "iso_3"],
    {"protectedAreasCount": "sum"},
)

DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name
100%|██████████| 14/14 [03:03<00:00, 13.12s/it]


DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name
 21%|██▏       | 3/14 [01:45<05:30, 30.01s/it]DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name
DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name
 57%|█████▋    | 8/14 [02:16<00:50,  8.40s/it]DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name
DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name
DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name
 79%|███████▊  | 11/14 [02:59<00:28,  9.46s/it]DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name
DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name
100%|█████████▉| 281/282 [00:20<00:02,  2.84s/it]

In [15]:
coverage = (
    final_data.pipe(calculate_global_area, ["year", "PA_DEF"], {"area": "sum"}, "iso_3")
    .pipe(separate_parent_iso, "iso_3")
    .pipe(add_region_iso, "iso_3")
    .replace(
        {
            "iso_3": {
                "ATA": "ABNJ",
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
                "GIB": "GBR",
            }
        }
    )
    .pipe(calculate_stats_cov, ["year", "PA_DEF"], "iso_3")
    .pipe(coverage_stats)
)

ProtectedAreaExtentSchema(
    coverage.pipe(
        output,
        "iso_3",
        {"PA_DEF": {"0": 2, "1": 1}},
        {"PARENT_NAME": "location", "PA_DEF": "protection_status"},
        ["area", "iso_3"],
    )
).to_csv(
    output_file,
    index=True,
)

In [ ]:
strapi.deleteCollectionData(strapi_collection, list(range(1, 300)))

In [ ]:
strapi.importCollectionData(
    strapi_collection,
    output_file,
)

### Mpa atlas - country stats Fully or highly protected

We are going to use the intermediate data from eez, in order to create a dataset that can be used as a land mask.
The steps are:
1. Load eez
2. Spatial inner Join the eez dataset with the Mpaatlas one
3. iso assign using the sovereign one provided by mpaatlas
4. dissolve by location
5. calculate the area for global regions and eez countries ussing mollwide projection
6. prepare the data to be ingested in strapi
7. upload the data to strapi

In [16]:
pipe = "mpaatlas"
strapi_collection = "mpaa-protection-level-stat"

pipe_dir_eez = FileConventionHandler("eez")
pipe_dir_mpaatlas = FileConventionHandler(pipe)
output_file = pipe_dir_mpaatlas.get_processed_step_path(current_step).joinpath(
    "mpaatlas_protection_level.csv"
)

# Download the EEZ file && unzip it
download_and_unzip_if_needed(pipe_dir_eez, prev_step, mysettings)
# Download the mpas file && unzip it
download_and_unzip_if_needed(pipe_dir_mpaatlas, prev_step, mysettings)

# Load the data
eez = gpd.read_file(pipe_dir_eez.get_step_fmt_file_path(prev_step, "shp")).pipe(clean_geometries)
mpaatlas_intermediate = gpd.read_file(
    pipe_dir_mpaatlas.get_step_fmt_file_path(prev_step, "shp")
).pipe(clean_geometries)

/home/mambauser/data/eez/processed/eez_preprocess.zip
/home/mambauser/data/eez/processed/preprocess
/home/mambauser/data/mpaatlas/processed/mpaatlas_preprocess.zip
/home/mambauser/data/mpaatlas/processed/preprocess


In [17]:
eez_mpaatlas_data_join = await spatial_join(
    eez, mpaatlas_intermediate.pipe(mpaatlas_filter_stablishment)
)

  0%|          | 0/282 [00:00<?, ?it/s]

<class 'shapely.geometry.base.GeometrySequence'>


100%|██████████| 282/282 [00:27<00:00,  4.06s/it]

100%|██████████| 282/282 [00:27<00:00, 10.21it/s]


<class 'shapely.geometry.base.GeometrySequence'>


In [40]:
# To get an idea of the spatial join results
# eez_mpaatlas_data_join.to_file(
#     pipe_dir_mpaatlas.get_processed_step_path(current_step).joinpath("mpaatlas_sjoin.shp"),
#     driver="ESRI Shapefile",
# )

In [18]:
eez_mpaatlas_data_join.dissolve(by=["protecti_1", "location_i"], aggfunc={"name": "count"}).reset_index().to_file(
pipe_dir_mpaatlas.get_processed_step_path(current_step).joinpath("mpaatlas_sjoin_dissolved.shp"),
driver="ESRI Shapefile",
)

DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name


In [19]:
result = (
    eez_mpaatlas_data_join.pipe(process_mpaatlas_data)
    .pipe(calculate_global_area, gby_col=["protecti_1"], iso_column="location_i")
    .pipe(separate_parent_iso)
    .replace(
        {
            "location_i": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="location_i")
    .pipe(calculate_stats, gby_col=["protecti_1"], iso_column="location_i")
    .pipe(fix_monaco, iso_column="location_i", area_column="area_km2")
    .pipe(
        output,
        iso_column="location_i",
        rep_d={
            "protecti_1": {
                "fully or highly protected": 1,
                "less protected or unknown": 2,
            }
        },
        rename={"protecti_1": "mpaa_protection_level", "area_km2": "area"},
        drop_cols=[],
    )
)

ProtectionLevelSchema(result[~result.location.isna()].assign(year=2023)).to_csv(
    output_file, index=True
)

DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name


In [ ]:
strapi.deleteCollectionData(strapi_collection, list(range(1, 300)))

In [ ]:
strapi.importCollectionData(
    strapi_collection,
    output_file,
)

### Protected seas  - fishing protection level

In [15]:
pipe = "protectedseas"
strapi_collection = "fishing-protection-level-stat"

pipe_dir = FileConventionHandler(pipe)
input_file = pipe_dir.get_processed_step_path(prev_step).joinpath("protectedseas_stats.xlsx")
output_file = pipe_dir.get_processed_step_path(current_step).joinpath("lfp.csv")

# Download the protected seas file && unzip it
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name="vizzuality_processed_data/protectedseas/preprocess/protectedseas_stats.xlsx",
    file=input_file,
    operation="r",
)

# Load the data
protectedseas_intermediate = pd.read_excel(input_file)

DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token


In [16]:
final = (
    protectedseas_intermediate.replace(
        {
            "iso_sov": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            },
            "lfp": {
                5: "highly",
                4: "highly",
                3: "moderately",
                2: "less",
                1: "less",
            },
        }
    )
    .pipe(
        calculate_global_area,
        gby_col=["lfp"],
        iso_column="iso_sov",
        agg_ops={"area_sqkm": "sum", "total_area": "max"},
    )
    .pipe(add_region_iso, iso_column="iso_sov")
    .pipe(
        calculate_stats,
        gby_col=["lfp"],
        ops={"area_sqkm": "sum", "total_area": "max"},
        iso_column="iso_sov",
    )
    .pipe(lambda x: x.assign(pct=round(x.area_sqkm / x.total_area, 2)))
    .pipe(
        output,
        iso_column="iso_sov",
        rep_d={
            "lfp": {
                "highly": 1,
                "moderately": 2,
                "less": 3,
            }
        },
        rename={"lfp": "fishing_protection_level", "area_sqkm": "area"},
        drop_cols=["iso_sov", "total_area"],
    )
)
FPLSchema(final[final.location.notna()]).to_csv(output_file, index=True)

/home/mambauser/src/pipelines/processors.py:685: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(rep_d)


In [ ]:
strapi.deleteCollectionData(strapi_collection, list(range(1, 300)))

In [ ]:
strapi.importCollectionData(
    strapi_collection,
    output_file,
)

### Country mpas detail table data

  1- lower case the columns   
2- separate location that its regime is in dispute or on join regime  
3- calcualte area for mpaatlas data  
4- rename columns for merge  
5- merge maaatlas and mpa data identifying the source  
6- identify child resources and set them as childs  
7- calculate bbox  
8- set child resources  
9- prepare output for batch export  
10- upload data to strapi  

In [4]:
pipe = "mpa"
strapi_collection_mpas = "mpa"

pipe_dir = FileConventionHandler(pipe)
pipe_dir_mpaatlas = FileConventionHandler("mpaatlas")
output_file_mpas = pipe_dir.get_processed_step_path(current_step).joinpath("mpa_detail.csv")

# Download the protected atlas file && unzip it
download_and_unzip_if_needed(pipe_dir, prev_step, mysettings)
# Download the mpaatlas file 
download_and_unzip_if_needed(pipe_dir_mpaatlas, prev_step, mysettings)

DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token


/home/mambauser/data/mpa/processed/mpa_preprocess.zip
/home/mambauser/data/mpa/processed/preprocess
/home/mambauser/data/mpaatlas/processed/mpaatlas_preprocess.zip
/home/mambauser/data/mpaatlas/processed/preprocess


PosixPath('/home/mambauser/data/mpaatlas/processed/preprocess')

In [12]:
# Load the data
mpa_intermediate = gpd.read_file(pipe_dir.get_step_fmt_file_path(prev_step, "shp")).pipe(
    clean_geometries
)
mpaatlas_intermediate = gpd.read_file(
    pipe_dir_mpaatlas.get_step_fmt_file_path(prev_step, "shp")
).pipe(clean_geometries)

In [13]:
init_table = (
    pd.concat(
        [
            (
                mpa_intermediate.pipe(columns_to_lower)
                .pipe(separate_parent_iso, iso_column="parent_iso")
                .rename(
                    columns={
                        "parent_iso": "iso",
                        "status_yr": "year",
                        "gis_m_area": "area_km2",
                        "status": "establishm",
                    }
                )
            ).assign(source="protected_planet"),
            (
                mpaatlas_intermediate.pipe(calculate_area)
                .pipe(extract_wdpaid_mpaatlas)
                .pipe(separate_parent_iso, iso_column="location_i")
                .rename(
                    columns={
                        "location_i": "iso",
                        "wdpa_id": "wdpa_pid",
                        "designatio": "desig_eng",
                    }
                )
            ).assign(source="mpaatlas"
            ).astype({"mpa_zone_i": "Int64"}),
        ],
        ignore_index=True,
    )
    .reset_index(drop=True)
    .replace(
        {
            "iso": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .sort_values(by=["wdpa_pid", "wdpa_pid", "source"], ascending=[True, True, False])
)

In [91]:
#  to be run if things change a lot in the future
# iucn_cat = pd.DataFrame(
#     {"slug": init_table.iucn_cat.dropna().unique(), "name": init_table.iucn_cat.dropna().unique()},
#     index=pd.Index(np.arange(1, len(init_table.iucn_cat.dropna().unique()) + 1)),
# )
# iucn_cat.to_csv(pipe_dir.get_processed_step_path(current_step).joinpath("iucn_categories.csv"), index=True)

iucn_cat = pd.read_csv(
    pipe_dir.get_processed_step_path(current_step).joinpath("iucn_categories.csv"), index_col=0
)

In [75]:
mpa_table = (
    init_table.pipe(add_bbox, "bbox")
    .pipe(define_is_child)
    .pipe(set_child_id)
    .sort_values(by=["wdpaid", "is_child"], ascending=[True, True])
    .reset_index(drop=True)
    .pipe(
        output,
        iso_column="iso",
        rep_d={
            "status": {
                "Adopted": 4,
                "implemented": 6,
                "Established": 6,
                "Designated": 5,
                "Proposed": 3,
                "Inscribed": 3,
                "unknown": 1,
            },
            "pa_def": {"0": 2, "1": 1},
            "year": {0: pd.NA},
            "iucn_cat": dict(
                iucn_cat[["slug"]]
                .reset_index(drop=False)
                .iloc[:, [1, 0]]
                .to_dict(orient="tight")["data"]
            ),
            "source": {"protected_planet": 3, "mpaatlas": 1},
            "protection": {
                "full": 3,
                "light": 4,
                "incompatible": 5,
                "high": 6,
                "minimal": 7,
                "unknown": 8,
                "unknown/to be determined": 8,
            },
            "establishm": {
                "actively managed": 4,
                "implemented": 6,
                "designated": 5,
                "Designated": 5,
                "proposed or committed": 3,
                "Proposed": 3,
                "Inscribed": 3,
                "Established": 5,
                "Adopted": 5,
                "unknown": 1,
            },
        },
        rename={
            "pa_def": "protection_status",
            "area_km2": "area",
            "iucn_cat": "mpa_iucn_category",
            "desig_eng": "designation",
            "protection": "mpaa_protection_level",
            "establishm": "mpaa_establishment_stage",
            "source": "data_source",
        },
        drop_cols=["geometry", "iso", "protecti_1"],  # "WDPAID",
    )
    .pipe(add_child_parent_relationship)
    .astype(
        {
            "year": "Int32",
            "mpa_iucn_category": "Int64",
            "protection_status": "Int64",
        }
    )
    .sort_index()
)

/home/mambauser/src/pipelines/processors.py:660: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  
/home/mambauser/src/pipelines/processors.py:685: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  """


In [76]:
mpa_table.to_csv(output_file_mpas, index=True)

In [80]:
# todo investigate the issue with area as null

In [79]:
batch_export(
    mpa_table[mpa_table.area.notna()],
    5000,
    MPAsSchema,
    pipe_dir.get_processed_step_path(current_step),
    "mpa_detail",
    format="json",
    strapi_colection=strapi_collection_mpas,
)

In [33]:
# # This code is to be able to identify groups that has wdpa_pid so in the future if needed we could combine the group geometries to generate a wdpa coverage geometry
# init_table[
#     (
#         init_table.sort_values(by=["wdpaid", "source"], ascending=[True, False])
#         .groupby("wdpaid")
#         .transform("size")
#         .gt(1)
#     )
#     & (init_table.wdpa_pid.str.extract(r"([A-Za-z]+)", expand=False).notna())
# ].groupby("wdpaid")
# .geometry.apply(lambda x: x.union_all())

#### upload data to strapi

In [126]:
strapi.deleteCollectionData("mpa", list(range(1, 18914)))

In [ ]:
for i in range(0, 4):
    strapi.importCollectionData(
        strapi_collection_mpas,
        mpa_folder.joinpath(f"mpa_join_mpatlas_prot_{i}.csv"),
    )